In [2]:
# HANDLING MISSING DATA
# --- None: Pythonic missing data ---
import numpy as np
import pandas as pd
vals1 = np.array([1, None, 3, 4], dtype=object)
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
# This dtype=object means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects
for dtype in ['object', 'int']:
 print("dtype =", dtype)
 %timeit np.arange(1E6, dtype=dtype).sum()
 print()

dtype = object
48.7 ms ± 884 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.36 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [5]:
# --- Nan: Missing  numerical data ---
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [8]:
#  Regardless of the operation, the result of arithmetic with NaN will be another NaN:
print(1 + np.nan)
print(0 * np.nan)
print(vals2.sum(), vals2.min(), vals2.max())

nan
nan
nan nan nan


In [9]:
# NumPy does provide some special aggregations that will ignore these missing values:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

In [10]:
# --- NaN and None in Pandas ---
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [11]:
# For types that don’t have an available sentinel value, Pandas automatically type-casts when NA values are present.
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

In [14]:
# OPERATING ON NULL VALUES
# --- Detecting Null Values ---
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

In [16]:
# --- Dropping null values ---
data.dropna()

0        1
2    hello
dtype: object

In [17]:
df = pd.DataFrame([[1, np.nan, 2],
                   [2, 3, 5],
                   [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [18]:
# By default, dropna() will drop all rows in which any null value is present:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [19]:
# Alternatively, you can drop NA values along a different axis; axis=1 drops all columns containing a null value
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [20]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [21]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [22]:
# For finer-grained control, the thresh parameter lets you specify a minimum number of non-null values for the row/column to be kept
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


In [23]:
# --- Filling null values ---
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [24]:
# We can fill NA entries with a single value, such as zero:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [25]:
# We can specify a forward-fill to propagate the previous value forward:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [26]:
# Or we can specify a back-fill to propagate the next values backward:
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [27]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [28]:
df.fillna(method='ffill', axis=1)
# Notice that if a previous value is not available during a forward fill, the NA value remains.

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
